In [ ]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [ ]:
# get data
data = pd.read_csv('Level_3b_NonPath_SP2_EsCo_PSL_1_2_3_4_574.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

         Xc1.A     Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0     2.566136  2.332851  ...        0.032370        0.030361
1    17.798460  6.396322  ...        0.032770        0.032138
2     5.387067  2.835298  ...        0.031862        0.027646
3    13.095425  1.636928  ...        0.030777        0.032884
4     9.634552  5.831440  ...        0.032222        0.031622
..         ...       ...  ...             ...             ...
569  16.638587  8.839250  ...        0.035352        0.034787
570  13.692819  4.471124  ...        0.031048        0.031582
571   4.115225  3.566528  ...        0.029097        0.031874
572  12.470586  6.495097  ...        0.030560        0.031774
573   4.793531  1.597844  ...        0.031451        0.031101

[574 rows x 150 columns]
0      1
1      1
2      1
3      1
4      1
      ..
569    4
570    4
571    4
572    4
573    4
Name: Class, Length: 574, dtype: category
Categories (4, int64): [1, 2, 3, 4]


In [ ]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [ ]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.2,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(150, 4 - 1) = 3 components.
  ChangedBehaviorWarn

In [ ]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(4 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 150)            91200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 128)            142848    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 4)                 260       
Total params: 283,716
Trainable params: 283,716
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/20
28/28 [==============================] - 38s 139ms/step - loss: 0.6619 - accuracy: 0.2680 - val_loss: 0.5600 - val_accuracy: 0.3741
Epoch 2/20
28/28 [==============================] - 1s 29ms/step - loss: 0.5517 - accuracy: 0.4289 - val_loss: 0.5153 - val_accuracy: 0.4354
Epoch 3/20
28/28 [==============================] - 1s 35ms/step - loss: 0.5049 - accuracy: 0.4651 - val_loss: 0.4672 - val_accuracy: 0.5612
Epoch 4/20
28/28 [==============================] - 1s 30ms/step - loss: 0.4631 - accuracy: 0.5429 - val_loss: 0.4144 - val_accuracy: 0.6293
Epoch 5/20
28/28 [==============================] - 1s 33ms/step - loss: 0.4201 - accuracy: 0.6099 - val_loss: 0.3609 - val_accuracy: 0.7891
Epoch 6/20
28/28 [==============================] - 1s 35ms/step - loss: 0.3613 - accuracy: 0.7385 - val_loss: 0.2934 - val_accuracy: 0.8605
Epoch 7/20
28/28 [==============================] - 1s 33ms/step - loss: 0.3161 - accuracy: 0.8064 - val_loss: 0.2331 - val_accuracy: 0.9014
Epoch 8/20


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.9421768707482994
test-acc = 0.9217687074829932
train-CFR =               precision    recall  f1-score   support

           0       0.94      0.86      0.90       217
           1       0.90      0.93      0.92       222
           2       0.98      0.99      0.99       215
           3       0.94      0.99      0.96       228

    accuracy                           0.94       882
   macro avg       0.94      0.94      0.94       882
weighted avg       0.94      0.94      0.94       882

test-CFR =               precision    recall  f1-score   support

           0       0.97      0.77      0.86        77
           1       0.82      0.96      0.88        72
           2       0.99      0.99      0.99        79
           3       0.93      0.98      0.96        66

    accuracy                           0.92       294
   macro avg       0.93      0.92      0.92       294
weighted avg       0.93      0.92      0.92       294

